In [2]:
import os
import sys
sys.path.append('/workspace/Documents')  ### remove this if not needed!
import numpy as np
import pandas as pd 
import nibabel as nb
import original_SAM.functions_collection as ff

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15]


In [3]:
original_path = '/mnt/camca_NAS/SAM_for_CMR/data/HFpEF_database/temporal'
new_path = '/mnt/camca_NAS/SAM_for_CMR/example_data_original_sam/data'

In [3]:
patient_id =  'ID_0015'
save_folder = os.path.join(new_path, patient_id)
ff.make_folder([save_folder])

# load data
img_file = nb.load(os.path.join(original_path, patient_id, '4d_temporal_img_nonzero.nii.gz'))
seg_file = nb.load(os.path.join(original_path, patient_id, '4d_temporal_seg_nonzero.nii.gz'))
affine = img_file.affine

img = img_file.get_fdata()[:,:,:,0]
print(img.shape)
seg = seg_file.get_fdata()[:,:,:,0]

(256, 256, 9)


In [4]:
# fine out the slices in seg with segmentation labels
slices_with_labels = np.unique(np.where(seg>0)[2])
print(slices_with_labels)
img = img[:,:,slices_with_labels]
seg = seg[:,:,slices_with_labels]

# save for each slice
for slice_n in range(img.shape[2]):
    img_slice = img[:,:,slice_n]
    seg_slice = seg[:,:,slice_n]
    nb.save(nb.Nifti1Image(img_slice, affine), os.path.join(new_path, patient_id, f'img_slice{slice_n:02d}.nii.gz'))
    nb.save(nb.Nifti1Image(seg_slice, affine), os.path.join(new_path, patient_id, f'seg_slice{slice_n:02d}.nii.gz'))

[0 1 2 3 4 5 6 7 8]


## make a patient spreadsheet

In [4]:
result = []
patient_list = ff.find_all_target_files(['ID*'],new_path)
print(patient_list)

patient_list_save_folder = os.path.join(os.path.dirname(new_path), 'Patient_list')
ff.make_folder([patient_list_save_folder])

for i in range(0,patient_list.shape[0]):
    current_patient = patient_list[i]
    patient_id = os.path.basename(current_patient)

    slice_list = ff.find_all_target_files(['img_slice*'],current_patient)
    for s in range(0,slice_list.shape[0]):
        slice_index = s
        img_file = os.path.join(current_patient, f'img_slice{slice_index:02d}.nii.gz')
        seg_file = os.path.join(current_patient, f'seg_slice{slice_index:02d}.nii.gz')
        result.append([patient_id, slice_index, img_file, seg_file])
        df = pd.DataFrame(result, columns=['patient_id', 'slice_index', 'img_file', 'seg_file'])
        df.to_excel(os.path.join(patient_list_save_folder, 'patient_list.xlsx'), index=False)

['/mnt/camca_NAS/SAM_for_CMR/example_data_original_sam/data/ID_0002'
 '/mnt/camca_NAS/SAM_for_CMR/example_data_original_sam/data/ID_0015']
